In [ ]:
#!pip install kaggle


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn.utils


In [ ]:
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae' )

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head()

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [ ]:
df.columns


Index(['country', 'year', 'country_code', 'record', 'crop_land',
       'grazing_land', 'forest_land', 'fishing_ground', 'built_up_land',
       'carbon', 'total', 'QScore'],
      dtype='object')

In [ ]:

df.QScore.unique()

array(['3A', '2A', '2B', '1A', '1B', nan], dtype=object)

In [ ]:
df.QScore.value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [ ]:
df.QScore.value_counts(normalize=True)

3A    0.713181
2A    0.146512
2B    0.139863
1A    0.000222
1B    0.000222
Name: QScore, dtype: float64

In [ ]:
df.shape

(72186, 12)

In [ ]:
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [ ]:
df= df.dropna()

In [ ]:
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [ ]:
df.shape

(51713, 12)

In [ ]:
df.QScore.value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

In [ ]:
df.QScore.replace('1A','2A',inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [ ]:
df_2A = df[df.QScore=='2A']
df_3A = df[df.QScore== '3A'].sample(350)
data_df = df_2A.append(df_3A)

In [ ]:
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape
data_df.QScore.value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

In [ ]:
 data_df= data_df.drop(columns=['country_code','country','year'])
 X = data_df.drop(columns='QScore')
 Y = data_df['QScore']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(X,Y,test_size=0.3,random_state=0)
y_train.value_counts()


3A    243
2A    170
Name: QScore, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record) 
x_test.record = encoder.transform(x_test.record)

In [ ]:
x_train.record

285    3
113    2
18     1
76     2
206    5
      ..
277    3
9      1
359    0
192    7
559    1
Name: record, Length: 413, dtype: int64

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_resample(x_train,y_train)

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=[ 'record' ])) 
normalised_train_df = pd.DataFrame(normalised_train_df, 
                                   columns=x_train_balanced.drop(columns=[ 'record' ]).columns) 
normalised_train_df[ 'record' ] = x_train_balanced[ 'record' ] 


In [ ]:
x_test= x_test.reset_index(drop=True)
normalised_test_df =scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df,
                                  columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

LOGISTIC REGRESSION


In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)

LogisticRegression(C=1.0,class_weight=None,dual=False,fit_intercept=True,
                   intercept_scaling=1, l1_ratio=1, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver= 'lbfgs' , tol= 0.0001 , 
                   verbose= 0,warm_start= False )

LogisticRegression(l1_ratio=1)

In [ ]:
from sklearn.metrics import recall_score, accuracy_score, precision_score,f1_score, confusion_matrix

In [ ]:
new_predictions=log_reg.predict(normalised_test_df)
new_predictions


array(['2A', '2A', '2A', '3A', '3A', '2A', '2A', '2A', '2A', '2A', '2A',
       '2A', '3A', '2A', '3A', '2A', '3A', '3A', '2A', '2A', '2A', '3A',
       '3A', '2A', '2A', '2A', '2A', '2A', '2A', '2A', '3A', '2A', '3A',
       '2A', '2A', '2A', '2A', '3A', '3A', '3A', '3A', '3A', '2A', '3A',
       '2A', '2A', '2A', '2A', '3A', '3A', '3A', '2A', '3A', '3A', '2A',
       '2A', '2A', '3A', '2A', '2A', '2A', '3A', '3A', '3A', '2A', '2A',
       '3A', '3A', '2A', '2A', '2A', '2A', '2A', '2A', '2A', '3A', '2A',
       '2A', '3A', '2A', '2A', '2A', '2A', '2A', '2A', '2A', '3A', '2A',
       '2A', '3A', '2A', '2A', '2A', '2A', '2A', '2A', '2A', '3A', '3A',
       '2A', '2A', '3A', '3A', '2A', '2A', '2A', '2A', '2A', '2A', '2A',
       '2A', '3A', '3A', '2A', '2A', '2A', '2A', '2A', '3A', '2A', '3A',
       '3A', '2A', '3A', '2A', '3A', '2A', '2A', '2A', '3A', '2A', '2A',
       '2A', '2A', '3A', '2A', '2A', '3A', '2A', '3A', '2A', '3A', '2A',
       '2A', '2A', '2A', '2A', '3A', '3A', '2A', '2

In [ ]:
cnf_mat=confusion_matrix(y_true = y_test, y_pred=new_predictions, labels=['2A','3A'])
cnf_mat

array([[49, 21],
       [70, 37]])

In [ ]:
from sklearn.model_selection import cross_val_score


scores = cross_val_score(log_reg,normalised_train_df,y_balanced, cv=5,scoring='f1_macro')
scores

array([0.47459207, 0.44181586, 0.47241647, 0.55193898, 0.54154229])

In [ ]:
accuracy=accuracy_score(y_true=y_test, y_pred=new_predictions)
print( 'Accuracy: {}' .format(round(accuracy*100), 2))
                    

Accuracy: 49


In [ ]:
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label ='2A')
print('Precision: {}'.format(round(precision*100),2))

Precision: 41


In [ ]:
recall = recall_score(y_true=y_test,y_pred=new_predictions, pos_label='2A')
print('Recall: {}'.format(round(precision*100),2))

Recall: 41


In [ ]:
f1= f1_score(y_true=y_test,y_pred=new_predictions, pos_label='2A')
print('F1: {}'.format(round(f1*100),2))

F1: 52


In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(normalised_train_df)
f1_scores=[]

In [ ]:
for train_index,test_index in kf.split(normalised_train_df):
  x_train,x_test= normalised_train_df.iloc[train_index],normalised_train_df.iloc[test_index]
  y_train,y_test = y_balanced[train_index], y_balanced[test_index]
  model = LogisticRegression().fit(x_train,y_train)
f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label ='2A')*100)

In [ ]:

from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=1)
f1_scores=[]

In [ ]:
for train_index,test_index in skf.split(normalised_train_df, y_balanced):
  x_train, x_test = np.array(normalised_train_df)[train_index], np.array(normalised_test_df)[test_index]
  y_train, y_test =y_balanced[train_index], y_balanced[test_index]
  model = LogisticRegression().fit(x_train, y_train)
f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A'))





IndexError: ignored

In [ ]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo,
 scoring= 'f1_macro' )
average_score = scores.mean() * 100 
